In [1]:
from siuba import *
import pandas as pd
import datetime as dt

import download_data
import conveyal_vars
import json

In [2]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

In [3]:
from calitp_data_analysis.tables import tbls

In [4]:
import zipfile
import numpy as np

In [5]:
from conveyal_vars import GCS_PATH

In [6]:
from shared_utils import gtfs_utils_v2

In [27]:
from calitp_data_analysis import get_fs
fs = get_fs()


# Exporting AC Transit Feeds

Ad-hoc for GenAI request July 2024, but maybe a useful example for the future.

"Get every unique version of a GTFS feed across a date range, in original GTFS zip format"

In [7]:
GCS_PATH

'gs://calitp-analytics-data/data-analyses/conveyal_update/'

In [8]:
feeds_on_target = gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(selected_date=conveyal_vars.TARGET_DATE)

In [9]:
feeds_on_target >> filter(_.name.str.contains('511 AC T'))

,key,date,feed_key,feed_timezone,base64_url,gtfs_dataset_key,name,regional_feed_type,type
180,0a1483f02bc6d02b38bd5ef7d106ec96,2023-10-18,d56477a57ef86563ebb8eda6ed509653,US/Pacific,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,c499f905e33929a641f083dad55c521e,Bay Area 511 AC Transit Schedule,Regional Subfeed,schedule


In [23]:
ac_23_feeds = (tbls.mart_gtfs.fct_scheduled_trips()
 >> filter(_.gtfs_dataset_key == 'c499f905e33929a641f083dad55c521e')
 >> filter(_.service_date > '2022-12-31')
 >> filter(_.service_date < '2024-01-01')
 >> distinct(_.feed_key, _.base64_url, _.service_date)
 >> collect()
 # >> head(1)
)

In [24]:
ac_23_feeds

,feed_key,base64_url,service_date
0,d56477a57ef86563ebb8eda6ed509653,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-10-25
1,026ed9b585ddcd860d59a367139d7429,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-04-27
2,026ed9b585ddcd860d59a367139d7429,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-04-04
3,d56477a57ef86563ebb8eda6ed509653,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-08-15
4,d56477a57ef86563ebb8eda6ed509653,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-09-04
...,...,...,...
269,af5d0cf0af941d169873cca44fedc3ad,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-07-04
270,d56477a57ef86563ebb8eda6ed509653,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-09-13
271,d56477a57ef86563ebb8eda6ed509653,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-08-13
272,af5d0cf0af941d169873cca44fedc3ad,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-07-29


In [25]:
df2 = ac_23_feeds >> distinct(_.feed_key, _keep_all=True)

In [26]:
df2

,feed_key,base64_url,service_date
0,d56477a57ef86563ebb8eda6ed509653,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-10-25
1,026ed9b585ddcd860d59a367139d7429,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-04-27
2,f63db5ffe22c8a846885a1f68ec7113b,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-12-03
3,af5d0cf0af941d169873cca44fedc3ad,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-08-03
4,3ed66c11d38d2d3e5a610c6d1b34ec2a,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-12-18
5,afabb460d8101af466bb52abdd6a4b71,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-05-19
6,a746bce335fb8477339a910e5ecfa2eb,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-08-06
7,84eb243b2456b2d4a0b57d4eefe31000,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-03-29
8,04bc8a2bf207378727371a9253eb12f2,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-06-11


In [30]:
def download_feed(row):
    # need wildcard for file too -- not all are gtfs.zip!
    uri = f'gs://calitp-gtfs-schedule-raw-v2/schedule/dt={row.service_date.strftime("%Y-%m-%d")}/*/base64_url={row.base64_url}/*.zip'
    print(uri)
    fs.get(uri, f'./{row.feed_key}_gtfs.zip')
    # fs.get(uri, f'./{row.gtfs_dataset_name.replace(" ", "_")}_{row.feed_key}_gtfs.zip')
    # print(f'downloaded {row.path}/{row.feed_key}_gtfs.zip')

In [31]:
df2.apply(download_feed, axis = 1)

gs://calitp-gtfs-schedule-raw-v2/schedule/dt=2023-10-25/*/base64_url=aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZWVkcz9vcGVyYXRvcl9pZD1BQw==/*.zip
gs://calitp-gtfs-schedule-raw-v2/schedule/dt=2023-04-27/*/base64_url=aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZWVkcz9vcGVyYXRvcl9pZD1BQw==/*.zip
gs://calitp-gtfs-schedule-raw-v2/schedule/dt=2023-12-03/*/base64_url=aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZWVkcz9vcGVyYXRvcl9pZD1BQw==/*.zip
gs://calitp-gtfs-schedule-raw-v2/schedule/dt=2023-08-03/*/base64_url=aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZWVkcz9vcGVyYXRvcl9pZD1BQw==/*.zip
gs://calitp-gtfs-schedule-raw-v2/schedule/dt=2023-12-18/*/base64_url=aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZWVkcz9vcGVyYXRvcl9pZD1BQw==/*.zip
gs://calitp-gtfs-schedule-raw-v2/schedule/dt=2023-05-19/*/base64_url=aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZWVkcz9vcGVyYXRvcl9pZD1BQw==/*.zip
gs://calitp-gtfs-schedule-raw-v2/schedule/dt=2023-08-06/*/base64_url=aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGF

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
dtype: object